In [2]:
import os
import numpy as np
from raug.metrics import Metrics, accuracy
from raug.utils.loader import get_labels_frequency
import pandas as pd

# Especificar o caminho para a pasta "best_results"
folder_path = "best_results"
_csv_path_test = "/home/a52550/Desktop/datasets/ISIC2017/test/ISIC-2017_Test_v2_Part3_GroundTruth.csv"
true_labels = pd.read_csv(_csv_path_test).sort_values(by=['image_id'])['category']

# Inicializar as variáveis para armazenar as previsões dos modelos
ensemble_preds_best = None
ensemble_preds_last = None
num_models = 0
ser_lab_freq = get_labels_frequency(_csv_path_test, "category", "image_id")
_labels_name = ser_lab_freq.index.values #np.reshape(ser_lab_freq.index.values, (ser_lab_freq.index.values.shape[0], 1))

df_best = pd.DataFrame()
df_last = pd.DataFrame()

# Percorrer as pastas de cada experimento
for experiment_folder in os.listdir(folder_path):

    if ".csv" in experiment_folder:
        continue

    print("Experimento: ", experiment_folder)

    _metric_options_best = {
        'save_all_path': os.path.join(folder_path, experiment_folder, 'ensemble_average/best'),
        'pred_name_scores': 'predictions.csv',
    }
    _metric_options_last = {
        'save_all_path': os.path.join(folder_path, experiment_folder, 'ensemble_average/last'),
        'pred_name_scores': 'predictions.csv',
    }


    experiment_path = os.path.join(folder_path, experiment_folder)
    metrics_best = Metrics (["accuracy", "topk_accuracy", "balanced_accuracy", "conf_matrix" "plot_conf_matrix", "precision_recall_report", "auc_and_roc_curve", "auc"] , _labels_name, _metric_options_best)
    metrics_last = Metrics (["accuracy", "topk_accuracy", "balanced_accuracy", "conf_matrix" "plot_conf_matrix", "precision_recall_report", "auc_and_roc_curve", "auc"] , _labels_name, _metric_options_last)

    ensemble_preds_best = None
    ensemble_preds_last = None

    model_folders = os.listdir(experiment_path)
    
    # Cria path para o experimento atual
    if 'ensemble_average' in model_folders:
        model_folders.remove('ensemble_average')
    else:
        os.mkdir(os.path.join(folder_path, experiment_folder, 'ensemble_average'))
        os.mkdir(os.path.join(_metric_options_best['save_all_path']))
        os.mkdir(os.path.join(_metric_options_last['save_all_path']))

    if 'ensemble_3_best_average' in model_folders:
        model_folders.remove('ensemble_3_best_average')

    try:

        # Percorrer as pastas de cada modelo
        for model_folder in model_folders:

            model_path = os.path.join(experiment_path, model_folder)

            # Carregar as previsões do modelo atual
            model_preds_path_best = os.path.join(model_path, "test_pred_best/predictions.csv")
            model_preds_path_last = os.path.join(model_path, "test_pred_last/predictions.csv")

            model_preds_best = pd.read_csv(model_preds_path_best).sort_values(by=['image'])
            model_preds_last = pd.read_csv(model_preds_path_last).sort_values(by=['image'])

            images_id = model_preds_best[['image']]
            model_preds_best = model_preds_best[['0','1','2']]
            model_preds_last = model_preds_last[['0','1','2']]

            # Adicionar as previsões do modelo atual ao ensemble
            if ensemble_preds_best is None:
                ensemble_preds_best = model_preds_best
            else:
                ensemble_preds_best += model_preds_best

            if ensemble_preds_last is None:
                ensemble_preds_last = model_preds_last
            else:
                ensemble_preds_last += model_preds_last

            num_models += 1

        # Calcular a média das previsões dos modelos
        ensemble_preds_best /= num_models
        ensemble_preds_last /= num_models

        metrics_best.update_scores(true_labels, ensemble_preds_best.to_numpy(), images_id.to_numpy())
        metrics_last.update_scores(true_labels, ensemble_preds_last.to_numpy(), images_id.to_numpy())

        metrics_best.compute_metrics()
        metrics_last.compute_metrics()

        metrics_best.save_scores()
        metrics_last.save_scores()

        folder_name = {'folder_name': experiment_folder}
        dict_best = {**folder_name, **metrics_best.metrics_values}
        dict_last = {**folder_name, **metrics_last.metrics_values}

        del dict_best['auc_and_roc_curve']
        del dict_best['precision_recall_report']

        del dict_last['auc_and_roc_curve']
        del dict_last['precision_recall_report']

        df_best = df_best.append(pd.DataFrame(dict_best, columns=dict_best.keys(), index=[0]), ignore_index=True)
        df_last = df_last.append(pd.DataFrame(dict_last, columns=dict_last.keys(), index=[0]), ignore_index=True)

    except Exception as e:
        print("Erro no experimento ", experiment_folder)
        print(e)


df_best.to_csv(os.path.join(folder_path, 'ensemble_average_best.csv'), index=False)
df_last.to_csv(os.path.join(folder_path, 'ensemble_average_last.csv'), index=False)

# Avaliar a precisão do modelo de ensemble
# true_classes_path = os.path.join(folder_path, "true_classes.csv")
# true_classes = np.loadtxt(_csv_path_test, delimiter=",", skiprows=1)

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# accuracy = accuracy_score(true_classes, ensemble_classes)
# precision = precision_score(true_classes, ensemble_classes, average="macro")
# recall = recall_score(true_classes, ensemble_classes, average="macro")
# f1 = f1_score(true_classes, ensemble_classes, average="macro")

# # Imprimir as métricas de avaliação
# print(f"Acurácia: {accuracy:.3f}")
# print(f"Precisão: {precision:.3f}")
# print(f"Recall: {recall:.3f}")
# print(f"F1-score: {f1:.3f}")


Experimento:  dull_razor_shades_of_gray_cropped_images_folder_2
Saving the scores in best_results/dull_razor_shades_of_gray_cropped_images_folder_2/ensemble_average/best
Saving the scores in best_results/dull_razor_shades_of_gray_cropped_images_folder_2/ensemble_average/last
Experimento:  DA0_noPP
Saving the scores in best_results/DA0_noPP/ensemble_average/best
Saving the scores in best_results/DA0_noPP/ensemble_average/last
Experimento:  DA0_PP
Saving the scores in best_results/DA0_PP/ensemble_average/best
Saving the scores in best_results/DA0_PP/ensemble_average/last
Experimento:  dull_razor
Saving the scores in best_results/dull_razor/ensemble_average/best
Saving the scores in best_results/dull_razor/ensemble_average/last
Experimento:  DA1_noPP
Saving the scores in best_results/DA1_noPP/ensemble_average/best
Saving the scores in best_results/DA1_noPP/ensemble_average/last
Experimento:  shades_of_gray_cropped_images_folder
Saving the scores in best_results/shades_of_gray_cropped_imag

/home/a52550/Desktop/masters/cnn/learning_cycle/benchmarks/isic/raug/utils/classification_metrics.py:347: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Saving the scores in best_results/non_local_means_cropped_images_folder/ensemble_average/best
Saving the scores in best_results/non_local_means_cropped_images_folder/ensemble_average/last
Experimento:  DA1_PPcropped
Saving the scores in best_results/DA1_PPcropped/ensemble_average/best
Saving the scores in best_results/DA1_PPcropped/ensemble_average/last
Experimento:  gaussian_filter_2
Saving the scores in best_results/gaussian_filter_2/ensemble_average/best
Saving the scores in best_results/gaussian_filter_2/ensemble_average/last
Experimento:  CLAHE_cropped_images_folder
Erro no experimento  CLAHE_cropped_images_folder
[Errno 2] No such file or directory: 'best_results/CLAHE_cropped_images_folder/efficientnet_b6_fold-1_lrinit-0.001_batchsize-4_optimizer-SGD_maxepochs-50_DA-None_PPen-CLAHE_PPha-None_PPco-None_PPde-None_PPno-True_PPcr-cropped_images_folder_PPre-True_16794346882817037/test_pred_best/predictions.csv'
Experimento:  dull_razor_cropped_images_folder
Saving the scores in best_

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [13]:
import os
import numpy as np
from raug.metrics import Metrics, accuracy
from raug.utils.loader import get_labels_frequency
import pandas as pd

# Especificar o caminho para a pasta "best_results"
folder_path = "best_results"
_csv_path_test = "/home/a52550/Desktop/datasets/ISIC2017/test/ISIC-2017_Test_v2_Part3_GroundTruth.csv"
true_labels = pd.read_csv(_csv_path_test).sort_values(by=['image_id'])['category']

# Inicializar as variáveis para armazenar as previsões dos modelos
ensemble_preds_best = None
ensemble_preds_last = None
num_models = 0
ser_lab_freq = get_labels_frequency(_csv_path_test, "category", "image_id")
_labels_name = ser_lab_freq.index.values #np.reshape(ser_lab_freq.index.values, (ser_lab_freq.index.values.shape[0], 1))

df_best = pd.DataFrame()
df_last = pd.DataFrame()

# Percorrer as pastas de cada experimento
for experiment_folder in os.listdir(folder_path):

    if ".csv" in experiment_folder:
        continue

    print("Experimento: ", experiment_folder)

    _metric_options_best = {
        'save_all_path': os.path.join(folder_path, experiment_folder, 'ensemble_3_best_average/best'),
        'pred_name_scores': 'predictions.csv',
    }
    _metric_options_last = {
        'save_all_path': os.path.join(folder_path, experiment_folder, 'ensemble_3_best_average/last'),
        'pred_name_scores': 'predictions.csv',
    }


    experiment_path = os.path.join(folder_path, experiment_folder)
    metrics_best = Metrics (["accuracy", "topk_accuracy", "balanced_accuracy", "conf_matrix" "plot_conf_matrix", "precision_recall_report", "auc_and_roc_curve", "auc"] , _labels_name, _metric_options_best)
    metrics_last = Metrics (["accuracy", "topk_accuracy", "balanced_accuracy", "conf_matrix" "plot_conf_matrix", "precision_recall_report", "auc_and_roc_curve", "auc"] , _labels_name, _metric_options_last)

    ensemble_preds_best = None
    ensemble_preds_last = None

    experiment_folders = os.listdir(experiment_path)
    
    # Cria path para o experimento atual
    if 'ensemble_average' in experiment_folders:
        experiment_folders.remove('ensemble_average')
    if 'ensemble_3_best_average' in experiment_folders:
        experiment_folders.remove('ensemble_3_best_average')
    else:
        os.mkdir(os.path.join(folder_path, experiment_folder, 'ensemble_3_best_average'))
        os.mkdir(os.path.join(_metric_options_best['save_all_path']))
        os.mkdir(os.path.join(_metric_options_last['save_all_path']))


    try:
        best_balanced_accuracy = 0
        best_metric_ensemble = None

        # Percorrer as pastas de cada modelo
        for i1 in range(len(experiment_folders)-1):
            for i2 in range(i1+1, len(experiment_folders)-1):
                for i3 in range(i2+1, len(experiment_folders)-1):
                    model_folders = [experiment_folders[i1], experiment_folders[i2], experiment_folders[i3]]

                    for model_folder in model_folders:

                        model_path = os.path.join(experiment_path, model_folder)

                        # Carregar as previsões do modelo atual
                        model_preds_path_best = os.path.join(model_path, "test_pred_best/predictions.csv")
                        model_preds_path_last = os.path.join(model_path, "test_pred_last/predictions.csv")

                        model_preds_best = pd.read_csv(model_preds_path_best).sort_values(by=['image'])
                        model_preds_last = pd.read_csv(model_preds_path_last).sort_values(by=['image'])

                        images_id = model_preds_best[['image']]
                        model_preds_best = model_preds_best[['0','1','2']]
                        model_preds_last = model_preds_last[['0','1','2']]

                        # Adicionar as previsões do modelo atual ao ensemble
                        if ensemble_preds_best is None:
                            ensemble_preds_best = model_preds_best
                        else:
                            ensemble_preds_best += model_preds_best

                        if ensemble_preds_last is None:
                            ensemble_preds_last = model_preds_last
                        else:
                            ensemble_preds_last += model_preds_last

                        num_models += 1

                    # Calcular a média das previsões dos modelos
                    ensemble_preds_best /= num_models
                    ensemble_preds_last /= num_models

                    metrics_best.update_scores(true_labels, ensemble_preds_best.to_numpy(), images_id.to_numpy())
                    metrics_last.update_scores(true_labels, ensemble_preds_last.to_numpy(), images_id.to_numpy())

                    metrics_best.compute_metrics()
                    metrics_last.compute_metrics()

                    # metrics_best.save_scores()
                    # metrics_last.save_scores()

                    if metrics_best.metrics_values['balanced_accuracy'] > best_balanced_accuracy:
                        best_balanced_accuracy = metrics_best.metrics_values['balanced_accuracy']
                        best_metric_ensemble = metrics_best
                        best_experiment = experiment_folder

                        ensemble_data = {
                            'folder_name': experiment_folder,
                            'ensemble_3_best': model_folders[0].split('fold')[0] + "_" + model_folders[1].split('fold')[0] + "_" + model_folders[2].split('fold')[0]
                        }
                        dict_best = {**ensemble_data, **metrics_best.metrics_values}
                        dict_last = {**ensemble_data, **metrics_last.metrics_values}

                        del dict_best['auc_and_roc_curve']
                        del dict_best['precision_recall_report']

                        del dict_last['auc_and_roc_curve']
                        del dict_last['precision_recall_report']

                    # df_best = df_best.append(pd.DataFrame(dict_best, columns=dict_best.keys(), index=[0]), ignore_index=True)
                    # df_last = df_last.append(pd.DataFrame(dict_last, columns=dict_last.keys(), index=[0]), ignore_index=True)

        best_metrics_ensemble.save_scores()

        df_best = df_best.append(pd.DataFrame(dict_best, columns=dict_best.keys(), index=[0]), ignore_index=True)
        df_last = df_last.append(pd.DataFrame(dict_last, columns=dict_last.keys(), index=[0]), ignore_index=True)

    except Exception as e:
        print("Erro no experimento ", experiment_folder)
        print(e.with_traceback())



df_best.to_csv(os.path.join(folder_path, 'ensemble_3_best_average_best.csv'), index=False)
df_last.to_csv(os.path.join(folder_path, 'ensemble_3_best_average_last.csv'), index=False)


Experimento:  dull_razor_shades_of_gray_cropped_images_folder_2


/home/a52550/Desktop/masters/cnn/learning_cycle/benchmarks/isic/raug/utils/classification_metrics.py:347: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [7]:
folders = os.listdir('best_results')

folders.remove('ensemble_average_best.csv')
folders.remove('ensemble_average_last.csv')

# Percorrer as pastas de cada experimento
for i1 in range(len(folders)):
    for i2 in range(i1+1,len(folders)):
        for i3 in range(i2+1,len(folders)):
            print(fi1, i2, i3)

0 1 2
0 1 3
0 1 4
0 1 5
0 1 6
0 1 7
0 1 8
0 1 9
0 1 10
0 1 11
0 1 12
0 1 13
0 1 14
0 1 15
0 1 16
0 1 17
0 1 18
0 1 19
0 1 20
0 1 21
0 1 22
0 1 23
0 1 24
0 1 25
0 1 26
0 1 27
0 1 28
0 1 29
0 1 30
0 1 31
0 1 32
0 1 33
0 1 34
0 1 35
0 1 36
0 1 37
0 2 3
0 2 4
0 2 5
0 2 6
0 2 7
0 2 8
0 2 9
0 2 10
0 2 11
0 2 12
0 2 13
0 2 14
0 2 15
0 2 16
0 2 17
0 2 18
0 2 19
0 2 20
0 2 21
0 2 22
0 2 23
0 2 24
0 2 25
0 2 26
0 2 27
0 2 28
0 2 29
0 2 30
0 2 31
0 2 32
0 2 33
0 2 34
0 2 35
0 2 36
0 2 37
0 3 4
0 3 5
0 3 6
0 3 7
0 3 8
0 3 9
0 3 10
0 3 11
0 3 12
0 3 13
0 3 14
0 3 15
0 3 16
0 3 17
0 3 18
0 3 19
0 3 20
0 3 21
0 3 22
0 3 23
0 3 24
0 3 25
0 3 26
0 3 27
0 3 28
0 3 29
0 3 30
0 3 31
0 3 32
0 3 33
0 3 34
0 3 35
0 3 36
0 3 37
0 4 5
0 4 6
0 4 7
0 4 8
0 4 9
0 4 10
0 4 11
0 4 12
0 4 13
0 4 14
0 4 15
0 4 16
0 4 17
0 4 18
0 4 19
0 4 20
0 4 21
0 4 22
0 4 23
0 4 24
0 4 25
0 4 26
0 4 27
0 4 28
0 4 29
0 4 30
0 4 31
0 4 32
0 4 33
0 4 34
0 4 35
0 4 36
0 4 37
0 5 6
0 5 7
0 5 8
0 5 9
0 5 10
0 5 11
0 5 12
0 5 13
0 5 14
0

In [2]:
folder_path = "best_results/ensemble_average_best.csv"

df = pd.read_csv(folder_path)

df.sort_values(by=['balanced_accuracy'], ascending=False)

,folder_name,accuracy,precision_macro,recall_macro,fscore_macro,support_macro,precision_micro,recall_micro,fscore_micro,support_micro,precision_weighted,recall_weighted,fscore_weighted,support_weighted,topk_accuracy,balanced_accuracy,auc
23,dull_razor_shades_of_gray_2,0.825000,0.768381,0.775779,0.764840,NaN,0.825000,0.825000,0.825000,NaN,0.828002,0.825000,0.822775,NaN,0.950000,0.775779,0.918206
14,DA2_noPP,0.798333,0.725435,0.770212,0.741383,NaN,0.798333,0.798333,0.798333,NaN,0.810963,0.798333,0.801174,NaN,0.946667,0.770212,0.910052
13,noDA_PPcropped_2,0.811667,0.747481,0.768994,0.752966,NaN,0.811667,0.811667,0.811667,NaN,0.814711,0.811667,0.810583,NaN,0.960000,0.768994,0.917954
26,noDA_noPP,0.801667,0.729930,0.768470,0.739910,NaN,0.801667,0.801667,0.801667,NaN,0.812021,0.801667,0.802170,NaN,0.950000,0.768470,0.914077
25,shades_of_gray_2,0.806667,0.739907,0.768450,0.749970,NaN,0.806667,0.806667,0.806667,NaN,0.810535,0.806667,0.806519,NaN,0.958333,0.768450,0.915933
32,noDA_PPcropped,0.801667,0.728983,0.767052,0.743718,NaN,0.801667,0.801667,0.801667,NaN,0.809983,0.801667,0.803544,NaN,0.955000,0.767052,0.913904
15,DA0_noPP_2,0.806667,0.736661,0.764740,0.747613,NaN,0.806667,0.806667,0.806667,NaN,0.810910,0.806667,0.807239,NaN,0.951667,0.764740,0.909936
7,DA0_noPP_3,0.806667,0.737530,0.763594,0.743997,NaN,0.806667,0.806667,0.806667,NaN,0.813454,0.806667,0.806801,NaN,0.961667,0.763594,0.910221
0,dull_razor_shades_of_gray_cropped_images_folder_2,0.801667,0.735253,0.761049,0.740352,NaN,0.801667,0.801667,0.801667,NaN,0.810266,0.801667,0.802080,NaN,0.951667,0.761049,0.911422
3,DA1_noPP,0.770000,0.694286,0.755501,0.711649,NaN,0.770000,0.770000,0.770000,NaN,0.797376,0.770000,0.776046,NaN,0.943333,0.755501,0.908595
